In [8]:
# Supervised learning using a feedforward network

In [9]:
# Get and preprocess the data according to our requirement
import pandas as pd
data = pd.read_csv('insurance.csv')
print('\n #Features = '+str(data.shape[1]))
print('\n #Samples = '+str(data.shape[0]))
data.head()


 #Features = 2

 #Samples = 63


,Number of claims,Payment
0,108,392.5
1,19,46.2
2,13,15.7
3,124,422.2
4,40,119.4


In [10]:
# Following is the code for the training network.
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
from numpy import random

torch.manual_seed(47)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from torch.nn.functional import normalize

dat = data.to_numpy(dtype= np.float32) # dataframe  to numpy 
# Split data into input and labels
x = torch.from_numpy(dat[:,[0]])
x = torch.nn.functional.normalize(x, p=2.0, dim = 0)
y = torch.from_numpy(dat[:,[1]])

# Split data into test and train
train_size = int(0.8*len(dat))
test_size = len(dat)- train_size
x_train, x_test = torch.utils.data.random_split(x, [train_size, test_size])
y_train, y_test = torch.utils.data.random_split(y, [train_size, test_size])

# define train and test Datasets
class CustomDataset_train(Dataset):
    def __init__(self):
        self.x_train = x_train
        self.y_train = y_train
        self.num_samples = len(x_train)
        
    def __getitem__(self, index):
        return self.x_train[index], self.y_train[index]

    def __len__(self):
        return self.num_samples

class CustomDataset_test(Dataset):

    def __init__(self):
        self.x_test = x_test
        self.y_test = y_test
        self.num_samples = len(x_test)

    def __getitem__(self, index):
        return self.x_test[index], self.y_test[index]

    def __len__(self):
        return self.num_samples

data_train = CustomDataset_train()
data_test = CustomDataset_test()

batch_size= 5 #set batch size

train_dataloader = DataLoader(dataset = data_train, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(dataset = data_test, batch_size=batch_size, shuffle=True)

In [11]:
import torch.nn as nn
input_size = x.shape[1]
output_size = y.shape[1]

# define network architecture
class myModel(nn.Module):
    def __init__(self,input_dim, output_dim, hidden_size1 =128, hidden_size2 = 128):
        super(myModel, self).__init__()
        #define network layers
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)
      
    def forward(self,x):
        
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        
        out = self.fc3(out)
        return out
    
model = myModel(input_size, output_size)
print(model)

myModel(
  (fc1): Linear(in_features=1, out_features=128, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=128, out_features=1, bias=True)
)


In [12]:
#define loss function, optimizer and hyperparameters
loss_fn = torch.nn.MSELoss()
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  
total_step = len(train_dataloader)

In [13]:
def train(dataloader, model, loss_fn, optimizer, batch_size):
    model.train()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 1 == 0:
            loss, current = loss.item(), (batch+1) * batch_size
            print(f"Train_loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0
    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
    test_loss /= num_batches
    print(f"Avg_Test_loss: {test_loss:>8f} \n")

epochs = 10
loss_train = []
loss_test = []
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, batch_size)
    test(test_dataloader, model, loss_fn)


Epoch 1
-------------------------------
Train_loss: 13885.284180  [    5/   50]
Train_loss: 50193.808594  [   10/   50]
Train_loss: 1267.078369  [   15/   50]
Train_loss: 2156.162109  [   20/   50]
Train_loss: 14770.377930  [   25/   50]
Train_loss: 14362.869141  [   30/   50]
Train_loss: 13700.504883  [   35/   50]
Train_loss: 6330.865723  [   40/   50]
Train_loss: 17924.349609  [   45/   50]
Train_loss: 6530.818848  [   50/   50]
Avg_Test_loss: 19056.734701 

Epoch 2
-------------------------------
Train_loss: 38804.339844  [    5/   50]
Train_loss: 6585.289062  [   10/   50]
Train_loss: 8667.189453  [   15/   50]
Train_loss: 8287.187500  [   20/   50]
Train_loss: 10626.510742  [   25/   50]
Train_loss: 1715.315430  [   30/   50]
Train_loss: 5992.055664  [   35/   50]
Train_loss: 9693.899414  [   40/   50]
Train_loss: 1436.408691  [   45/   50]
Train_loss: 7567.862305  [   50/   50]
Avg_Test_loss: 11981.234843 

Epoch 3
-------------------------------
Train_loss: 3096.894531  [    5/